In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule

## 16

Построим граф всех победных комбинаций выбранного игрока настольной игры Okiya.  
Комбинации поражения в граф включать не будем, для экономии памяти.  
У каждой комбинации на своем ходе, будет вес, соответствующий количеству победных комбинаций в направлении финального хода.  
Так мы сможем сортировать комбинации хода, для выбора наиболее богатой потенциальными победами комбинации.

В игре два игрока. У каждого игрока свой граф побед.  
Размер поля 4 на 4 клетки.  
Возможно 16 ходов, по 8 на каждого игрока.  
Первым ходит игрок 0, Вторым игрок 1.

### win_combination

Для определения победной комбинации удобно использовать побитовый оператор И.  
Например, десятичное 51865 это двоичное 1100101010011001. Если разрезать на 4 части получится комбинация:  
1100  
1010  
1001  
1001  
Вертикальная линия слева - победная комбинация.  
Побитовый И действует как оператор умножения, умножая 1й бит слева на 1й бит справа и так каждый бит.  
Например, 111&010 вернёт 010. Это свойство позволяет использовать оператор И как фильтр.  
Что бы определить есть ли полный ряд единицы в левой колонке, применим оператор И на нашем числе 1100101010011001 и шаблоне 1000100010001000.  
В десятичном представлении это будет записано так: 51865&34952  
Если результат будет равен 34952 значит ряд единиц есть. В противном случае нули затрут одну или несколько единиц в колонке и в результате будет уже другое число.

In [2]:
51865&32768

32768

Опишем условие для каждой строки, каждой колонки и для квадратных победных комбинаций.

In [3]:
ker = SourceModule("""
__device__ bool win_combination(int i)
{
    int win[17]={
    61440, // row 0
    3840,  // row 1
    240,   // row 2
    15,    // row 3
    34952, // col 0
    17476, // col 1
    8738,  // col 2
    4369,  // col 3
    52224, // quad 0
    26112, // quad 1
    13056, // quad 2
    3264,  // quad 3
    1632,  // quad 4
    816,   // quad 5
    204,   // quad 6
    102,   // quad 7
    51     // quad 8
    };
    for (int a=0;a<17;a++) if ((i&win[a])==win[a]) return(true);    
    return(false);
}

__global__ void last_turn(float *outvec)
{
    int ones = 0;
    int a = 0;
    
    // Идентификатор блока-потока
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    
    // Убедимся, что число единиц и нулей в комбинации одинаково
    for(a=0;a<16;a++) if ((((int)round(pow(2,a)))&i)>0) ones++;
    
    // Проверим, победна ли комбинация среди возможных
    if (ones==8 && win_combination(i)) outvec[i] = 1;
}
""")

### last_turn

In [4]:
[1 if i%2 else 0 for i in range(16)]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

Последним ходит игрок 1. Определим все возможные, победные комбинации для текущего игрока на этапе последнего хода.  
Для поля 4 на 4 возможно 2**(4 * 4) что соответствует 65536 комбинаций.  
Используем 65536 потоков GPU. Каждый поток - уникальная комбинация поля.  
256 блоков в 256 сетках.
Каждая комбинация предствалена в виде int числа от 0 до 65536. Для представления в виде фишек на поле, переведем его в bin, разделим последовательность чисел на 4 части, разместив сверху вниз, например:  
0 1 0 1  
0 1 0 1  
0 1 0 1  
0 1 0 1  

In [5]:
combination_check_gpu = ker.get_function("last_turn")

combinations = np.zeros(256*256).astype(np.float32)
combinations_gpu_out = gpuarray.to_gpu(combinations)

%time combination_check_gpu( combinations_gpu_out, block=(256,1,1), grid=(256,1,1))

CPU times: user 589 µs, sys: 236 µs, total: 825 µs
Wall time: 747 µs


In [6]:
out_combinations = combinations_gpu_out.get()

Количество победных комбинаций:

In [7]:
np.count_nonzero(out_combinations)

6315

Посмотрим, на некоторые победные комбинации.

In [8]:
def bin_interpretation(i):
    clear_bin = bin(i)[2:]
    lead_zeros = ''.join(['0' for z in range(16-len(clear_bin))])
    return lead_zeros+clear_bin

In [9]:
np.where(out_combinations == 1)[0]

array([  255,   383,   447, ..., 65088, 65152, 65280])

In [10]:
for i in np.where(out_combinations == 1)[0][:3]:
    row = []
    i_bin = bin_interpretation(i)
    row.append(i_bin[:4])
    row.append(i_bin[4:8])
    row.append(i_bin[8:12])
    row.append(i_bin[12:])
    print('\n')
    for r in row:
        print(r)



0000
0000
1111
1111


0000
0001
0111
1111


0000
0001
1011
1111


## 15

Настало время перейти к 15 ходу.

In [11]:
[1 if i%2 else 0 for i in range(16)]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

Ход 0-го игрока.  
В этот ход:  
- Отсутствует один жетон 1-го игрока, что порождает 4 * 4 = 16 ответвлений от каждой победной комбинации  
- Место отсутствующего жетона занимает одна из 16 карточек игры, что пораждает 16 вариантов каждого ответвления.  

Так, нам потребуется 6315 * 16 * 16 потоков что бы отметить победные комбинации на 15 ход.

In [12]:
comb_count = 6315
empty_count = 16
token_count = 16
comb_count * empty_count * token_count

1616640

In [13]:
comb_count * empty_count * token_count / 1024

1578.75

In [14]:
1616640/6315/16/16

1.0

Опытным путем установлено, что GPU калькулятор 1080ti может за 1 раз заполнить массив размером 2 147 450 900 (1024 блоков и 2097120 сеток). А теоретически, максимальный размер сетки X (MAX_GRID_DIM_X): 2 147 483 647. Запас солидный.  
Перечислим все, что в этот ход надо учесть.  
- От жетона будет зависить, куда следующий игрок сможет положить свою фишку  
- Победа противника = поражение игрока. Теперь надо учитывать победы не только одного игрока, но и второго, в отдельном графе.  
Заполним 15 слой всеми вариантами, исключив невозможные.  
Договоримся о структуре данных. Поле стало сложнее. Теперь значение каждой клетки может принимать значение от 0 до 18:  
- 0: первый игрок  
- 1: второй игрок  
- 2-18: один из 16 жетонов игры
Значит у нас набор 18-ричных значений. Посчитаем, сколько это десятичных значений, если не исключать поражения.
Для 1го игрока смысл имеют только ходы, победившие в 16 слое. Все ходы, на более низких слоях, могут быть без победы. Мы считаем лишь все их варианты.

Для вычисления победных комбинаций 15 хода, передадим в ядро три числа:  
- Победная комбинация 16 хода: 1 из 6315 значений  
- Позиция клетки жетона игры: 1 из 16  
- Номер жетона игры: 1 из 16  

Для передачи в ядро, закодируем эти числа в одном десятичном числе, разбив его на группы в двоичном представлении.

In [15]:
print( bin(6315), bin(16), bin(16) )

0b1100010101011 0b10000 0b10000


In [16]:
len('1100010101011'), len('10000'), len('10000')

(13, 5, 5)

In [17]:
int(''.join(['1' for i in range(13+5+5)]),2)

8388607

Всего 8 миллионов. Это нам вполне по силам. Опишем функцию распаковки

In [18]:
def unzip(full_int):
    part_a = full_int>>(5+5)
    part_b = full_int>>5&int('11111',2)
    part_c = full_int&int('11111',2)
    return part_a, part_b, part_c

# zip
full_bin = '1000000000010'+'10010'+'10100'
full_int = int(full_bin, 2)
print(full_bin, full_int)
# unzip
part_a, part_b, part_c = unzip(full_int)
print(bin(part_a), part_a)
print(bin(part_b), part_b)
print(bin(part_c), part_c)

10000000000101001010100 4196948
0b1000000000010 4098
0b10010 18
0b10100 20


Теперь в каждом ядре мы будем знать, к какой комбинации 16го хода оно относится, какая клетка свободна, какая на клетке фишка игры.  
Начиная с 15 хода мы не будем удалять проигрышные комбинации. Но нам предстоит снижать их вес.
Определим, сколько сеток нам понадобится на 1024 блока.  
Известно что максимальный набор комбинаций содержит в бинарном представлении 23 разряда:

In [19]:
13+5+5, int(''.join(['1' for i in range(13+5+5)]),2)

(23, 8388607)

Используем максимум блоков: 1024. Сколько сеток потребуется для описания чаисла 8388607 с 1024 блоками?

In [20]:
8388607/1024

8191.9990234375

Округлим в большую сторону

In [21]:
print(8192, 'Сетки потребуется для охвата', 8192*1024, 'комбинаций при', 1024, 'блоках')

8192 Сетки потребуется для охвата 8388608 комбинаций при 1024 блоках


Данные:  
- Смещения комбинаций 16 хода: int[6315]
- Веса комбинаций 16 хода: int[1616640]
- Смещения комбинаций 15 хода: int[1616640]
- Веса комбинаций 15 хода: int[1616640] # Количество комбинаций 15-го хода: 6315 * 16 * 16 = 1616640  

Алгоритм: 
- Вес 15 хода копируется с веса 16 хода
- Если комбинация в разрешенном интервале и победна для игрока 1, добавляем вес

In [22]:
bias_16 = np.where(out_combinations == 1)[0]
len(bias_16)

6315

In [23]:
weight_16 = np.ones(len(bias_16)).astype(np.float32)

In [24]:
weight_15 = np.zeros(6315 * 16 * 16)

In [25]:
bias_15 = np.zeros(6315 * 16 * 16)

In [30]:
ker = SourceModule("""
__device__ bool win_combination(int i)
{
    int win[17]={
    61440, // row 0
    3840,  // row 1
    240,   // row 2
    15,    // row 3
    34952, // col 0
    17476, // col 1
    8738,  // col 2
    4369,  // col 3
    52224, // quad 0
    26112, // quad 1
    13056, // quad 2
    3264,  // quad 3
    1632,  // quad 4
    816,   // quad 5
    204,   // quad 6
    102,   // quad 7
    51     // quad 8
    };
    for (int a=0;a<17;a++) if ((i&win[a])==win[a]) return(true);    
    return(false);
}

__global__ void turn(float *bias_16, float *weight_16, float *bias_15, float *weight_15)
{   
    // Идентификатор блока-потока
    int i = threadIdx.x + blockIdx.x * blockDim.x;
}
""")

/tmp/ipykernel_38320/515248658.py:1: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(31): warning #177-D: variable "i" was declared but never referenced


  ker = SourceModule("""


In [31]:
turn_15 = ker.get_function("turn")

In [34]:
bias_16_gpu = gpuarray.to_gpu(bias_16)
weight_16_gpu = gpuarray.to_gpu(weight_16)
bias_15_gpu = gpuarray.to_gpu(bias_15)
weight_15_gpu = gpuarray.to_gpu(weight_15)

%time combination_check_gpu(bias_16_gpu,weight_16_gpu,bias_15_gpu,weight_15_gpu,block=(1024,1,1), grid=(8192,1,1))

CPU times: user 67 µs, sys: 86 µs, total: 153 µs
Wall time: 158 µs
